# ML Python Code Generation using Causal Language Model

**In this Notebook, we aim to construct a Causal Language Model designed specifically for Python code generation. The main focus is to create a model capable of taking a fragment of Python code as input and generating the entire code sequence as output. The significance of this approach lies in its potential to streamline the coding process and enhance efficiency.**

**Due to the vast landscape of Python libraries and frameworks, we'll narrow down the scope of our model to cater specifically to Data Science libraries. This decision is driven by the desire to optimize resources and reduce processing time. Therefore, our model will be tailored to four essential Data Science libraries:**

- **Pandas**: A powerful library for data manipulation and analysis, providing versatile data structures and tools.

- **Matplotlib**: A widely-used library for creating static, interactive, and animated visualizations in Python.

- **Seaborn**: Built on top of Matplotlib, this library facilitates the creation of attractive statistical graphics.

- **Scikit-Learn**: An invaluable library for machine learning tasks, offering a wide array of algorithms and tools.

**By focusing on these libraries, we can harness the capabilities of the Causal Language Model to deliver efficient and accurate Python code generation, specifically tailored to Data Science tasks. Let's dive into the implementation and explore the potential of this novel approach!"**

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
#Install the necessary libraries
!pip install -q transformers
!pip install -q datasets

In [3]:
#Set the seed value
SEED = 4243

## Dataset

**Regarding the dataset used for training the Causal Language Model, the ideal candidate would be the [Codeparrot dataset](https://huggingface.co/datasets/transformersbook/codeparrot). However, due to its substantial size of nearly 180GB and its demanding computational requirements, we will opt for a more manageable version of this dataset. A [smaller](https://huggingface.co/datasets/huggingface-course/codeparrot-ds-train), curated version is conveniently accessible, and it will sufficiently serve our purposes.**

**This curated dataset is designed to provide a representative sample of Python code from GitHub repositories. While it may not encompass the entirety of the original dataset, it still contains diverse and relevant code examples. This scaled-down version helps conserve computational resources and facilitates a smoother and faster training process.**

**By utilizing this reduced dataset, we can still achieve substantial learning outcomes and generate Python code effectively. So, let's proceed with this more manageable dataset and explore the potentials of our Causal Language Model in Python code generation for Data Science libraries!**

In [4]:
#Download the dataset
from datasets import load_dataset, DatasetDict

train = load_dataset(path="huggingface-course/codeparrot-ds-train", split="train")
valid = load_dataset(path="huggingface-course/codeparrot-ds-valid", split="validation")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/huggingface-course--codeparrot-ds-train-a9b1bc4c2b855d04/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/huggingface-course--codeparrot-ds-valid-e5ece22bd7b6a6ac/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


In [5]:
#Combine the train and valid dataset into a single DatasetDict object
dataset = DatasetDict(
        {
        "train": train.shuffle(seed=SEED).select(range(5_000)),
        "valid": valid.shuffle(seed=SEED).select(range(5_00))
        }
                     )
#Training the whole data would require more computation than available
# GPU hours. So, we'll restrict the data to 5K samples only

In [6]:
#Have a look at the dataset fields
dataset

DatasetDict({
    train: Dataset({
        features: ['repo_name', 'path', 'copies', 'size', 'content', 'license'],
        num_rows: 5000
    })
    valid: Dataset({
        features: ['repo_name', 'path', 'copies', 'size', 'content', 'license'],
        num_rows: 500
    })
})

In [7]:
#Interview the data / Have a look at some samples
for key, value in dataset["train"][0].items():
    print(f">>>> {key}: {value[:1_000]}\n")
#Since the content field contains long codes, we have restrcit it to 1,000 characters

>>>> repo_name: paladin74/neural-network-animation

>>>> path: matplotlib/axis.py

>>>> copies: 10

>>>> size: 80644

>>>> content: """
Classes for the ticks and x and y axis
"""
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import six

from matplotlib import rcParams
import matplotlib.artist as artist
from matplotlib.artist import allow_rasterization
import matplotlib.cbook as cbook
import matplotlib.font_manager as font_manager
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
import matplotlib.scale as mscale
import matplotlib.text as mtext
import matplotlib.ticker as mticker
import matplotlib.transforms as mtransforms
import matplotlib.units as munits
import numpy as np
import warnings

GRIDLINE_INTERPOLATION_STEPS = 180


class Tick(artist.Artist):
    """
    Abstract base class for the axis ticks, grid lines and labels

    1 refers to the bottom of the plot for xticks and the left for yticks
 

## Data Preprocessing

**In the data preprocessing phase, we need to perform two essential steps to prepare the text data for the Causal Language Model: tokenization and data collation.**

#### Tokenization:
It involves breaking down the raw text (Python code in this case) into smaller units called tokens. These tokens could be individual words, subwords, or characters, depending on the specific requirements of the model. Once tokenized, the next step is to convert these tokens into numerical representations. This conversion enables the model to understand and process the data effectively, as neural networks work with numerical inputs.

#### Data Collation:
After tokenization, the data needs to be organized into sequences that the Causal Language Model can use for training. Since models typically learn from fixed-length sequences, we create data samples by sliding a window over the tokenized text. This means we extract consecutive token sequences from the code, making each sequence a training instance. By using this approach, the model can learn patterns and dependencies within the code, which is crucial for accurate Python code generation.

**Together, these two preprocessing steps pave the way for successful training of the Causal Language Model, enabling it to effectively generate Python code based on the patterns it learns from the prepared tokenized and collated data.**

In [8]:
#Instantiate the tokenizer
from transformers import AutoTokenizer

context_length = 128
tokenizer = AutoTokenizer.from_pretrained("huggingface-course/code-search-net-tokenizer")

**Before tokenizing the whole dataset, it is a good practice to check it on the sample inputs.**

In [9]:
#Let's tokenzie first 2 samples
samp_token = tokenizer(
    dataset["train"][:2]["content"],
    truncation=True,
    max_length=context_length,
    return_overflowing_tokens=True,
    return_length=True,
)

In [10]:
#Check the fields in the tokenized object
samp_token.keys()

dict_keys(['input_ids', 'attention_mask', 'length', 'overflow_to_sample_mapping'])

In [11]:
print(f"Input IDs length: {len(samp_token['input_ids'])}")
print(f"Input chunk lengths: {(samp_token['length'])}")
print(f"Chunk mapping: {samp_token['overflow_to_sample_mapping']}")

Input IDs length: 167
Input chunk lengths: [128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 21, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 105]
Chunk mapping: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

**Since the tokenizer works fine on the sample data, we can apply it on the whole dataset using Dataset.map() method**

In [12]:
#Define a function to tokenzie the whole dataset
def tokenize_ftn(element):
    outputs = tokenizer(
        element["content"],
        truncation=True,
        max_length=context_length,
        return_overflowing_tokens=True,
        return_length=True,
    )
    input_batch = []
    
    #Discard the tokens that are smaller than the context size
    for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
        if length == context_length:
            input_batch.append(input_ids)
    return {"input_ids": input_batch}

In [13]:
#Check the tokenization function on some sample input
for sample in dataset["train"]:
    tokenized_sample = tokenize_ftn(sample)
    break
tokenized_sample.keys()

dict_keys(['input_ids'])

In [14]:
#Apply the tokenization function on the whole dataset
tokenized_dataset = dataset.map(function=tokenize_ftn,
                                batched=True,
                                remove_columns=dataset["train"].column_names)
tokenized_dataset

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 139434
    })
    valid: Dataset({
        features: ['input_ids'],
        num_rows: 14567
    })
})

## Model

**In our pursuit of generating Python code with utmost proficiency, we will harness the capabilities of the GPT-2 language model. Having undergone extensive training on an extensive corpus of text, GPT-2 has developed a remarkable aptitude for comprehending and predicting patterns within the provided data. To tailor the model to our specific needs, we can fine-tune a pre-trained GPT-2 model.**

**Nonetheless, given the substantial amount of data available, a prudent approach would be to train the model from scratch. This allows us to customize the model's understanding of Python code and adapt it precisely to the intricacies of our dataset. Training from scratch ensures that the model becomes intimately familiar with the specific nuances and complexities of Python syntax, thereby optimizing its code generation capabilities.**

In [15]:
#Load the configuration file for the GPT-2 model
from transformers import AutoTokenizer, TFGPT2LMHeadModel, AutoConfig

#Load the configuration file for the GPT-2 model
config = AutoConfig.from_pretrained(pretrained_model_name_or_path="gpt2",
                                    vocab_size=len(tokenizer),
                                    n_ctx=context_length,
                                    bos_token_id=tokenizer.bos_token_id,
                                    eos_token_id=tokenizer.eos_token_id,
)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [16]:
#Print the configuration file
config

GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 0,
  "embd_pdrop": 0.1,
  "eos_token_id": 0,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 128,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.30.2",
  "use_cache": true,
  "vocab_size": 50000
}

In [17]:
#Instantiate the model with the configuration file
model = TFGPT2LMHeadModel(config=config)
model(model.dummy_inputs)  # Builds the model
model.summary()

Model: "tfgpt2lm_head_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer (TFGPT2MainLaye  multiple                 124242432 
 r)                                                              
                                                                 
Total params: 124,242,432
Trainable params: 124,242,432
Non-trainable params: 0
_________________________________________________________________


In [18]:
#Instantiate the data collator
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer,
                                                mlm=False,
                                                return_tensors="tf")

In [19]:
#Check the data collator on some samples
collated_sample = data_collator([tokenized_dataset["train"][i] for i in range(5)])
for key in collated_sample:
    print(f"{key} shape: {collated_sample[key].shape}")

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


input_ids shape: (5, 128)
attention_mask shape: (5, 128)
labels shape: (5, 128)


In [20]:
#Create tf.data.Dataset object
tf_train_dataset = model.prepare_tf_dataset(tokenized_dataset["train"],
                                            collate_fn=data_collator,
                                            shuffle=True,
                                            batch_size=32)

tf_eval_dataset = model.prepare_tf_dataset(tokenized_dataset["valid"],
                                           collate_fn=data_collator,
                                           shuffle=False,
                                           batch_size=32)

In [21]:
#Log in to the HuggingFace account
from huggingface_hub import notebook_login
notebook_login()

In [22]:
#Define the optimizer
from transformers import create_optimizer
import tensorflow as tf

num_train_epochs = 5
num_train_steps = len(tf_train_dataset) * num_train_epochs
optimizer, schedule = create_optimizer(init_lr=5e-5,
                                       num_warmup_steps=1_000,
                                       num_train_steps=num_train_steps,
                                       weight_decay_rate=0.01)

In [23]:
#Complie the model
model.compile(optimizer=optimizer)

In [25]:
#Define the callbacks
from transformers.keras_callbacks import PushToHubCallback

callback = PushToHubCallback(output_dir="python-code-generator",
                             tokenizer=tokenizer)

Cloning https://huggingface.co/MUmairAB/python-code-generator into local empty directory.


In [26]:
#train the model
model.fit(tf_train_dataset,
          validation_data=tf_eval_dataset,
          callbacks=[callback],
          epochs=5)

Epoch 1/5
4357/4357 [==============================] - 2917s 664ms/step - loss: 4.8998 - val_loss: 3.6062
Epoch 2/5
4357/4357 [==============================] - 2948s 677ms/step - loss: 3.3262 - val_loss: 3.0415
Epoch 3/5
4357/4357 [==============================] - 2944s 676ms/step - loss: 2.8622 - val_loss: 2.7799
Epoch 4/5
4357/4357 [==============================] - 2949s 677ms/step - loss: 2.6127 - val_loss: 2.6525
Epoch 5/5
4357/4357 [==============================] - 2910s 668ms/step - loss: 2.4821 - val_loss: 2.6055
